In [1]:
import pandahouse 
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats 
import matplotlib.pyplot as plt 
from scipy.stats import norm, ttest_ind
from tqdm import tqdm
%matplotlib inline

In [68]:
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator'
}
'2024-05-24'  '2024-05-30'

In [69]:
q = """
select COUNT(DISTINCT user_id) as total_number
from simulator_20240620.feed_actions 
where toDate(time) between '2024-06-21' and '2024-06-27'
"""

users_number = pandahouse.read_clickhouse(q, connection=connection)

In [70]:
# Размер выборки 
# Количество всех пользователей за неделю/2
users_number.total_number/2

0    34463.0
Name: total_number, dtype: float64

In [71]:
rng = np.random.default_rng()

In [72]:
# Распределения просмотров 
q = """
select views, count() as users
from (select  
    user_id,
    sum(action = 'view') as views
from simulator_20240620.feed_actions 
where toDate(time) between '2024-06-21' and '2024-06-27'
group by user_id
)
group by views
order by views
"""


views_distribution = pandahouse.read_clickhouse(q, connection=connection)

In [73]:
views_distribution['p'] = views_distribution['users']/views_distribution.users.sum()

In [74]:
# Распределения  пользовательские CTR
q = """
select 
   floor(ctr, 2) as ctr, count() as users
from (select toDate(time) as dt, 
    user_id,
    sum(action = 'like')/sum(action = 'view') as ctr
from simulator_20240620.feed_actions 
where toDate(time) between '2024-06-21' and '2024-06-27'
group by dt, user_id
)
group by ctr
"""

ctr_distribution = pandahouse.read_clickhouse(q, connection=connection)

In [75]:
ctr_distribution['p'] = ctr_distribution['users']/ctr_distribution.users.sum()

In [76]:
p_values_t = []

In [77]:
#они же через цикл
for _ in tqdm(range(20000)):
    # сэмплируем данные просмотров на основе распределения просмотров 
    group_A_views = rng.choice(views_distribution['views'].astype(np.int64), size=34463, replace=True, p=views_distribution['p'])
    group_B_views = rng.choice(views_distribution['views'].astype(np.int64), size=34463, replace=True, p=views_distribution['p'])
    
    # сэмплируем данные ctr на основе распределения ctr 
    group_A_ctr = rng.choice(ctr_distribution['ctr'], size=34463, replace=True, p=ctr_distribution['p'])
    group_B_ctr = rng.choice(ctr_distribution['ctr'], size=34463, replace=True, p=ctr_distribution['p'])
    
    # дабавляем эффект от алгоритма к нашим просмотрам
    group_B_views = group_B_views + ((1 + rng.binomial(n=1, p=0.5, size=34463)) \
                                 * rng.binomial(n=1, p=0.9, size=34463) \
                                 * (group_B_views >= 50))
    group_B_views = group_B_views.astype("int64")
    
    # симуляции биномиального распределения 
    clicks_A = rng.binomial(group_A_views, group_A_ctr)
    clicks_B = rng.binomial(group_B_views, group_B_ctr) 
    
    # собираем все значения p value в строку 
    p_values_t.append(stats.ttest_ind(clicks_A/group_A_views, clicks_B/group_B_views, equal_var=False).pvalue)

100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [08:57<00:00, 37.18it/s]


In [80]:
np.mean(np.array(p_values_t) < 0.05) * 100

4.8950000000000005

In [64]:
sum_of_p_t * 100

4.26196251470254

In [65]:
sum_of_p_m * 100

4.261810031981848

In [66]:
def power_calc(sample_size, views_df, ctr_df, threshold, nsim=1000):
    """
    Считает мощность для t-теста при заданных параметрах распределения и эффекта
    :param sample_size: размер выборок
    :param views_df: датафрейм со значениями просмотров и их вероятностями
    :param ctr_df: датафрейм со значениями прользовательских CTR и их вероятностями
    :param threshold: порог просмотров, после которого начинает действовать алгоритм
    :param nsim: количество симуляций (по умолчанию 1000)
    :return: значение мощности от 0 до 1
    """

    pvalues = [] #список, куда мы будем складывать p-value
    rng = np.random.default_rng() #генератор случайных чисел

    #симуляируем и сравниваем выборки nsim раз:

    for _ in tqdm(range(nsim)):

        #симулируем просмотры
        group_A_views = rng.choice(a=views_df.views, size=sample_size, replace=True, p=views_df.p).astype("int64")
        group_B_views = rng.choice(a=views_df.views, size=sample_size, replace=True, p=views_df.p).astype("int64")

        #симулируем эффект в тестовой группе
        group_B_views = group_B_views + (rng.binomial(n=1, p=0.9, size=sample_size) * (1 + rng.binomial(n=1, p=0.5, size=sample_size)) * (group_B_views >= threshold))
        group_B_views = group_B_views.astype("int64")

        #симулируем CTR
        group_A_ctr = rng.choice(a=ctr_df.ctr, size=sample_size, replace=True, p=ctr_df.p)
        group_B_ctr = rng.choice(a=ctr_df.ctr, size=sample_size, replace=True, p=ctr_df.p)

        #симулируем лайки
        clicks_A = rng.binomial(n=group_A_views, p=group_A_ctr)
        clicks_B = rng.binomial(n=group_B_views, p=group_B_ctr)

        #сравниваем t-тестом и сохраняем p-value
        pvalues.append(ttest_ind(clicks_A, clicks_B, equal_var=False).pvalue)

    #возвращаем мощность
    return np.mean(np.array(pvalues) < 0.05)

In [67]:
power_calc(sample_size=34463, views_df=views_distribution, ctr_df=ctr_distribution, threshold=50, nsim=20000)

  4%|██▊                                                                           | 725/20000 [00:18<08:17, 38.78it/s]


KeyboardInterrupt: 